### 작업형1 예시문제
- 참고 사이트
- https://www.datamanim.com/dataset/03_dataq/main_p1.html
- https://blog.naver.com/statstorm/222364100877
- https://rfriend.tistory.com/tag/sklearn..preprocessing..minmax_scale%28%29

#### 1-1
mtcars 데이터셋(mtcars.csv)의 qsec 컬럼을 최소최대 척도(Min-Max Scale)로 변환한 후 0.5보다 큰 값을 가지는 레코드 수를 구하시오.

In [1]:
import pandas as pd
df = pd.read_csv('../data/mtcars.csv')
df.head(3)

,Unnamed: 0,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
0,Mazda RX4,21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4
1,Mazda RX4 Wag,21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4
2,Datsun 710,22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1


In [11]:
df['qsec'].describe()

count    32.000000
mean     17.848750
std       1.786943
min      14.500000
25%      16.892500
50%      17.710000
75%      18.900000
max      22.900000
Name: qsec, dtype: float64

In [70]:
# 1. 최소값, 최대값을 구해서 '0~1'로 변환 (array로 안바꾸고)
qsec = df['qsec']
#qsec = df.loc[:,'qsec']
qsec_minmax = (qsec - qsec.min())/(qsec.max() - qsec.min())
Num = len(qsec_minmax[qsec_minmax>0.5])
print('record :', Num)

record : 9


In [17]:
# 최소값, 최대값을 구해서 '0~1'로 변환 (array로 바꿔서)
import numpy as np
X = np.array(df['qsec'])
X

array([16.46, 17.02, 18.61, 19.44, 17.02, 20.22, 15.84, 20.  , 22.9 ,
       18.3 , 18.9 , 17.4 , 17.6 , 18.  , 17.98, 17.82, 17.42, 19.47,
       18.52, 19.9 , 20.01, 16.87, 17.3 , 15.41, 17.05, 18.9 , 16.7 ,
       16.9 , 14.5 , 15.5 , 14.6 , 18.6 ])

In [62]:
X_MinMax = (X - X.min())/(X.max()-X.min())
overNum = len(X_MinMax[X_MinMax > 0.5])
print('record :', overNum)

record : 9


In [79]:
# 2. minmax_scale 사용
from sklearn.preprocessing import minmax_scale
X_MinMax_Scale = minmax_scale(X,axis=0,copy=True) #?
#X_MinMax_Scale
print('record :', len(X_MinMax_Scale[X_MinMax_Scale>=0.5]))

record : 9


#### 1-2
mtcars 데이터셋(mtcars.csv)의 qsec 컬럼을 표준정규분포 데이터 표준화 (standardization) 변환 후 최대, 최소값을 각각 구하시오.

In [95]:
qsec = df['qsec']
mean = qsec.mean()
std = qsec.std()
z = (qsec-mean)/std
z = np.array(z)
#z
Min = z.min()
Max = z.max()
print('Min :',Min, 'Max :',Max)

Min : -1.8740102832334835 Max : 2.826754592962484


#### 1-3
*mtcars 데이터셋(mtcars.csv)의 wt 컬럼의 이상치(IQR 1.5 외부에 존재하는)값들을 outlier 변수에 저장하라

In [20]:
import numpy as np

wt = df['wt']
q25, q75 = np.percentile(wt.values, [25, 75])
IQR = q75 - q25

outlier = wt[(wt > q75 + IQR * 1.5) | (wt < q25 - IQR * 1.5)].values
outlier

array([5.25 , 5.424, 5.345])

#### 이상치탐지삭제 함수만들기 
https://hwi-doc.tistory.com/entry/IQR-방식을-이용한-이상치-데이터Outlier-제거

In [ ]:
import numpy as np

def get_outlier(df=None, column=None, weight=1.5):
  # target 값과 상관관계가 높은 열을 우선적으로 진행
  quantile_25 = np.percentile(df[column].values, 25)
  quantile_75 = np.percentile(df[column].values, 75)

  IQR = quantile_75 - quantile_25
  IQR_weight = IQR*weight
  
  lowest = quantile_25 - IQR_weight
  highest = quantile_75 + IQR_weight
  
  outlier_idx = df[column][ (df[column] < lowest) | (df[column] > highest) ].index
  return outlier_idx

# 함수 사용해서 이상치 값 삭제
oulier_idx = get_outlier(df=df, column='A', weight=1.5)
df.drop(outlier_idx, axis=0, inplace=True)

#### 1-4
mtcars 데이터셋에서 mpg변수와 나머지 변수들의 상관계수를 구하여 다음과 같이 내림차순 정렬하여 표현하라

In [25]:
# mpg와 모든 변수의 상관계수 구하기
df.corr()[['mpg']]

,mpg
mpg,1.000000
cyl,-0.852162
disp,-0.847551
hp,-0.776168
drat,0.681172
wt,-0.867659
qsec,0.418684
vs,0.664039
am,0.599832
gear,0.480285


In [29]:
# 내림차순으로 정렬
df.corr()[['mpg']].sort_values(by = 'mpg' ,ascending = False)

,mpg
mpg,1.000000
drat,0.681172
vs,0.664039
am,0.599832
gear,0.480285
qsec,0.418684
carb,-0.550925
hp,-0.776168
disp,-0.847551
cyl,-0.852162


#### 1-5
mtcars 데이터셋에서 mpg변수를 제외하고 데이터 정규화 (standardscaler) 과정을 진행한 이후 PCA를 통해 변수 축소를 하려한다. 누적설명 분산량이 92%를 넘기기 위해서는 몇개의 주성분을 선택해야하는지 설명하라

In [44]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
pca_df = df.iloc[:,2:]
pca_df = sc.fit_transform(pca)
#pca_df


In [49]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
printcipalComponents = pca.fit_transform(pca_df)
principalDf = pd.DataFrame(data = printcipalComponents, columns = ['comp1', 'comp2'])

In [50]:
principalDf.head()

,comp1,comp2
0,0.632134,1.739877
1,0.605027,1.554343
2,2.801549,-0.122632
3,0.259204,-2.364265
4,-2.032508,-0.774822


In [43]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
pcaDf = df.iloc[:,2:]
pcaDf = scaler.fit_transform(pcaDf)
#pcaDf

In [40]:
from sklearn.decomposition import PCA

componentsNum=10
pca = PCA(n_components=componentsNum) 
printcipalComponents = pca.fit_transform(pcaDf)
principalDf = pd.DataFrame(data=printcipalComponents, columns = ['component'+str(x) for x in range(componentsNum)])

componentDf = pd.DataFrame(pca.explained_variance_ratio_,columns=['cumsumVariance']).cumsum().reset_index()
componentDf['index'] +=1
componentDf=componentDf.rename(columns={'index':'componentsCount'})